### Search Engine With Tools and Agents

In [2]:
## Arxiv-Research
## tools creation
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper


In [ ]:
## used the in-built tool of wikipedia
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki_tool.name

'wikipedia'

In [6]:
##
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)
arxiv_tool = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
arxiv_tool.name

'arxiv'

In [7]:
tools = [wiki_tool, arxiv_tool]
## to run this tools we require an LLM model and chat prompt template and then will be used this

#### Making our Own custom Tools[RAG Tool]

In [13]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings # instead i'm using OLLama embeddings
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [14]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200).split_documents(docs)
vectorstore = FAISS.from_documents(documents, OllamaEmbeddings(model="llama3",))
retriever = vectorstore.as_retriever()
retriever


VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000025A061ADF90>, search_kwargs={})

In [16]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever, "Langsmith-search", "Search any information about langsmith")
retriever_tool.name


'Langsmith-search'

In [17]:
tools = [wiki_tool, arxiv_tool, retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\Azhar\\langchain\\projects\\Q-A-chatbot-keeping-message-history-WebApp-using-streamlit\\.venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='Langsmith-search', description='Search any information about langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000025A6FC88550>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_commun

In [19]:
### Run all this tools with agents and LLm Models

## Tools, LLm--> AgentExecutor

from langchain_groq import ChatGroq
from dotenv import load_dotenv
import openai
load_dotenv()
import os
groq_api_key = os.getenv('GROQ_API_KEY')
openai.api_key = os.getenv('OPENAI_API_KEY')
llm = ChatGroq(groq_api_key=groq_api_key, model_name= "Llama3-8b-8192")



In [20]:
## Prompt template 
from langchain import hub
prompt = hub.pull("hwchasse17/openai-functions-agent")
prompt.messages

LangSmithNotFoundError: Resource not found for /commits/hwchasse17/openai-functions-agent/latest. HTTPError('404 Client Error: Not Found for url: https://api.smith.langchain.com/commits/hwchasse17/openai-functions-agent/latest', '{"detail":"Repo hwchasse17/openai-functions-agent not found"}')

In [ ]:
## agents 
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm,tools, prompt)


In [ ]:
### for running this agent we make
## agent executor
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools, verbose=True)
agent_executor

In [ ]:
agent_executor.invoke({"input": "Tell me about langsmith"})
